In [196]:
import numpy as np

## No eavesdropper

In [197]:
def random_bits(n):
    """
    This function tends to randomly generate n bits sequence for Alice to send
    """
    return np.random.randint(2, size = n)

def random_bases(n):
    """
    Generate a random sequence of n bases, where 0 represents the computational basis and 1 represents the Fourier basis.
    """
    return np.random.randint(2, size=n)

def encode_bits(bits, bases):
    """
    Encode bits based on the selected bases. 
    Z basis (0) does not change the bit.
    X basis (1) encodes 0 as |+> and 1 as |->.
    Here we simply represent |+> as 0 and |-> as 1 for simplicity in binary.
    """
    encoded = []
    for bit, basis in zip(bits, bases):
        if basis == 1:  # Fourier basis
            encoded.append(0 if bit == 0 else 1)  # |+> as 0, |-> as 1
        else:  # Computational basis
            encoded.append(bit)
    return encoded

def measure_bits(encoded, bob_bases):
    """
    Bob measures the encoded bits based on his choice of bases.
    If the bases match, he gets the correct bit.
    """
    measured = []
    for enc_bit, basis in zip(encoded, bob_bases):
        if basis == 1:  # Bob measures in Fourier basis
            measured.append(enc_bit)  # Simple model: assume perfect measurement
        else:  # Bob measures in Computational basis
            measured.append(enc_bit)  # Random outcome
    return measured

def extract_key(bits, bases, bob_bases):
    """
    Extract the shared key from bits where the bases match.
    """
    key = []
    for bit, alice_base, bob_base in zip(bits, bases, bob_bases):
        if alice_base == bob_base:
            key.append(bit)
    return key
    

In [198]:
def bb84_simulation(n_bits):
    alice_bits = random_bits(n_bits)
    alice_bases = random_bases(n_bits)
    encoded_bits = encode_bits(alice_bits, alice_bases)

    bob_bases = random_bases(n_bits)
    bob_measurements = measure_bits(encoded_bits, bob_bases)

    alice_key = extract_key(alice_bits, alice_bases, bob_bases)
    bob_key = extract_key(bob_measurements, alice_bases, bob_bases)

    return alice_key, bob_key

n_bits = 4096
alice_key, bob_key = bb84_simulation(n_bits)
print("Alice's Key:", alice_key)
print("Bob's Key:  ", bob_key)
print("Length: ", len(alice_key))

Alice's Key: [1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0,

## With eavesdropper

### Comparing Length

In [199]:
def eve_intercept_measure(encoded, bases):
    """Simulate Eve intercepting and measuring the quantum bits."""
    eve_bits = []
    for bit, basis in zip(encoded, bases):
        # Assuming Eve has a 50% chance to correctly measure the qubit
        if np.random.random() < 0.5:
            eve_bits.append(bit)  # Correct measurement
        else:
            eve_bits.append(1 - bit)  # Incorrect measurement introduces error
    return eve_bits

def compare_keys(alice_key, bob_key):
    """Randomly compare parts of the keys to detect eavesdropping."""
    indices = np.where(np.random.random(size=len(alice_key)) < 0.5)[0]
    matches = sum(1 for i in indices if alice_key[i] == bob_key[i])
    return matches, len(indices)

def bb84_with_eavesdropping(n_bits):
    alice_bits = random_bits(n_bits)
    alice_bases = random_bases(n_bits)
    encoded_bits = encode_bits(alice_bits, alice_bases)

    eve_bases = random_bases(n_bits)
    eve_measurements = eve_intercept_measure(encoded_bits, eve_bases)

    bob_bases = random_bases(n_bits)
    bob_measurements = measure_bits(eve_measurements, bob_bases)

    alice_key = extract_key(alice_bits, alice_bases, bob_bases)
    bob_key = extract_key(bob_measurements, alice_bases, bob_bases)

    matches, total_compared = compare_keys(alice_key, bob_key)
    return alice_key, bob_key, matches, total_compared

In [200]:
# Parameters
n_bits = 4096

# Run the simulation
alice_key, bob_key, matches, total_compared = bb84_with_eavesdropping(n_bits)
print(f"Matching keys in sample: {matches}/{total_compared}")

Matching keys in sample: 508/1016


### Faled detection probability

In [201]:
def bb84_with_eavesdropping(n_bits):
    alice_bits = random_bits(n_bits)
    alice_bases = random_bases(n_bits)
    encoded_bits = encode_bits(alice_bits, alice_bases)

    eve_bases = random_bases(n_bits)
    eve_measurements = eve_intercept_measure(encoded_bits, eve_bases)

    bob_bases = random_bases(n_bits)
    bob_measurements = measure_bits(eve_measurements, bob_bases)

    alice_key = extract_key(alice_bits, alice_bases, bob_bases)
    bob_key = extract_key(bob_measurements, alice_bases, bob_bases)

    matches, total_compared = compare_keys(alice_key, bob_key)
    return matches == total_compared, total_compared

def simulate_bb84(n_bits, n_simulations):
    undetected = 0
    for _ in range(n_simulations):
        perfectly_matched, _ = bb84_with_eavesdropping(n_bits)
        if perfectly_matched:
            undetected += 1
    return undetected / n_simulations

In [217]:
n_bits = 16
n_simulations = 10000

# Probability of not detecting an eavesdropper
probability_undetected = simulate_bb84(n_bits, n_simulations) / 2
print(f"Probability of not detecting an eavesdropper: {probability_undetected:.4f}")

Probability of not detecting an eavesdropper: 0.0601


In [206]:
np.power(0.5, 4)

0.0625